In [1]:
from models import *
from jepa import JEPA
from datasets import H5Dataset, SparseDataset, METADATA
from trainer import Trainer

ModuleNotFoundError: No module named 'hdf5plugin'

In [ ]:
mode = 'cite'
days = [2]

train_dataset = H5Dataset('train', mode, days=days)
train_dataloader = train_dataset.get_dataloader(batch_size=16)

val_dataset = H5Dataset('val', mode, days=days)
val_dataloader = val_dataset.get_dataloader(batch_size=16)

In [ ]:
latent_dim = 1024
n_chan = 256

in_encoder = Encoder(22050, latent_dim, 256, 6, 11, 'enformer', 'attention')
out_decoder = Decoder(latent_dim, 140, 256, 6, 11, 'enformer', 'conv')
predictor = LinearCoder([latent_dim, latent_dim], 256, True, 0.05)
models = {'in_encoder': in_encoder,
         'out_decoder': out_decoder,
         'predictor': predictor}
jepa = JEPA(models)
print(jepa)

In [ ]:
# example pipeline for JEPA inference
for (x, day), y in train_dataloader:  # grab one batch
    x, day = x, day
    y = y
    break 
print(jepa.infer(x, day).shape, y.shape)  # these better be the same shape lol

In [ ]:
initial_lrs = {'in_encoder': 0.04,
             'out_decoder': 0.04,
             'predictor': 0.04}
lr_decay_periods = {'in_encoder': 5,
             'out_decoder': 5,
             'predictor': 5}
lr_decay_gammas = {'in_encoder': 0.5,
             'out_decoder': 0.5,
             'predictor': 0.5}
weight_decays = {'in_encoder': 0.0001,
             'out_decoder': 0.0001,
             'predictor': 0.0001}
num_epochs = 20
eval_every = 1
patience = 2
num_tries = 1

trainer = Trainer(jepa=jepa, 
                  train_dataloader=train_dataloader, 
                  val_dataloader=val_dataloader, 
                  initial_lrs=initial_lrs,
                  lr_decay_gammas=lr_decay_gammas,
                  lr_decay_periods=lr_decay_periods,
                  weight_decays=weight_decays)
jepa = trainer.train(num_epochs, eval_every, patience, num_tries)

In [ ]:
for (x, day), y in train_dataloader:
    out = jepa.infer(x, day)
    print(out[0])
    print(y[0])
    break